# Assignment 2: Jetty Scheduling

Name: **Moritz Grävinghoff, Frederic Weiss, Christian Husmann**\
Data of submission: **01.04.2024**

## Question 3

Heuristic Approach: Decomposition by ships

### Import all required packages

In [1]:
import pandas as pd
from docplex.mp.model import Model
from itertools import product
from plotly import figure_factory as ff
import colorsys

### 1. Load data

In [2]:
instance = pd.ExcelFile(f'../instances/inst4.xlsx')

ship_names = instance.parse(f'Problem_4', header=1, usecols='G').iloc[:,0].to_list()
categories = instance.parse(f'Problem_4', header=1, usecols='H').iloc[:,0].to_list()
arrivals = instance.parse(f'Problem_4', header=1, usecols='K').iloc[:,0].to_list()

# Sort ships by arrival time ascending and alphabetically, if they have the same arrival time.
df_ships = pd.DataFrame({'name':ship_names,'category':categories,'arrival':arrivals}).sort_values(['arrival','name'], ascending=True).reset_index(drop=True)

# Override old lists, now in correct order
ship_names = df_ships['name'].tolist()
ships = list(range(len(ship_names)))
# As we start indexing by 0, decrease categories and arrivals by 1
categories = [x-1 for x in df_ships['category']]
arrivals = [x-1 for x in df_ships['arrival']]

# Read unloading times
unloading_times = instance.parse(f'Problem_4', header = 1, usecols = 'B:E').dropna()
unloading_times = unloading_times.values.tolist()

# Define stations
stations = list(range(4))

### 2. Define decomposition parameters

In [15]:
# Auxiliary parameters for rolling planning horizon
num_ships = len(ships)
window = 7
overlap = 4

### 3. Run decomposed problem

In [18]:
#-----------------------------------------------
# Create model instance
#-----------------------------------------------

model = Model(name='jetty_scheduling_decomposition')

#-----------------------------------------------
# Define decision variable
#-----------------------------------------------

X = model.integer_var_dict(product(ships,stations), name='starting') # first period that ship s is located on station p
W = model.integer_var_dict(product(ships,stations), name='waiting') # periods that ship s waits at station p before moving to next station
A = model.binary_var_dict(product(ships,stations), name='assignment') # 1 if ship s is assigned to station p for unloading, 0 otherwise
E = model.binary_var_dict(product(ships,ships), name='order') # 1 if ship s enters the jetty before ship s_bar, 0 otherwise
F = model.integer_var_dict(ships, name='finish') # finishing time of ship s

#-----------------------------------------------
# Define constraints only for the ship window 
#-----------------------------------------------

bigM = 9999

# (1) each ship has to be assigned to exactly one station for unloading
model.add_constraints(model.sum(A[s,p] for p in stations) == 1 for s in ships)

# (2) no movement to first station before arrival
model.add_constraints(X[s,0] >= arrivals[s] for s in ships)

# (3) stay at assigned station for unloading, else for at least one period
model.add_constraints(X[s,p+1] == X[s,p] + A[s,p] * unloading_times[categories[s]][p] + (1-A[s,p]) + W[s,p] for s in ships for p in stations if p<3)

# (4) move to next station only if previous ship has already left
model.add_constraints(X[s_bar,p] >= X[s,p] + A[s,p] * unloading_times[categories[s]][p] + (1-A[s,p]) + W[s,p] - E[s_bar,s] * bigM for s in ships for s_bar in ships for p in stations if s!=s_bar)

# (5) ship is either before or after every other ship
model.add_constraints(1 == E[s,s_bar] + E[s_bar,s] for s in ships for s_bar in ships if s!=s_bar)

# (6) finishing time of each ship
model.add_constraints(F[s] == X[s,3] + A[s,3] * unloading_times[categories[s]][3] + (1-A[s,3]) for s in ships)

#-----------------------------------------------
# Decomposition
#-----------------------------------------------

for first in range(0,num_ships,overlap):

    # find last ship that is included in this window
    last = first + window - 1
    if last > num_ships - 1:
        last = num_ships - 1

    ships_window = ships[first:last+1]

    #-----------------------------------------------
    # Define objective function 
    #-----------------------------------------------
    
    # Minimize total sum of periods all ships are in waiting position or at the jetty
    model.minimize(model.sum(F[s] - arrivals[s] for s in ships[:last+1]))

    #-----------------------------------------------
    # Solve the model
    #-----------------------------------------------

    model.solve()
    
    #-----------------------------------------------
    # Fix the decision variables except for overlap
    #-----------------------------------------------

    model.add_constraints(X[s,p] == X[s,p].solution_value for s in ships_window[:overlap] for p in stations)
    model.add_constraints(W[s,p] == W[s,p].solution_value for s in ships_window[:overlap] for p in stations)
    model.add_constraints(A[s,p] == A[s,p].solution_value for s in ships_window[:overlap] for p in stations)

#### 4. Export results to Excel file

In [19]:
def export_results():

    return 

### 2. Create Gantt chart

#### Excursion

Extend the original plotly color list to be able to display 20 ships in instance 4.

In [20]:
def generate_similar_colors(hex_colors, saturation_factor=.5, lightness_factor=.8):
    new_colors = []
    
    for hex_color in hex_colors:
        # Convert hex to RGB
        rgb_color = tuple(int(hex_color[i:i+2], 16) / 255.0 for i in (1, 3, 5))
        
        # Convert RGB to HLS
        h, l, s = colorsys.rgb_to_hls(*rgb_color)
        
        # Adjust saturation and lightness
        s *= saturation_factor
        l *= lightness_factor
        
        # Convert back to RGB
        new_rgb_color = tuple(round(c * 255) for c in colorsys.hls_to_rgb(h, l, s))
        
        # Convert RGB to hex
        new_hex_color = "#{:02x}{:02x}{:02x}".format(*new_rgb_color)
        
        new_colors.append(new_hex_color)
    
    return new_colors

# Original plotly list of colors
original_colors = [
    '#1f77b4',  # muted blue
    '#ff7f0e',  # safety orange
    '#2ca02c',  # cooked asparagus green
    '#d62728',  # brick red
    '#9467bd',  # muted purple
    '#8c564b',  # chestnut brown
    '#e377c2',  # raspberry yogurt pink
    '#7f7f7f',  # middle gray
    '#bcbd22',  # curry yellow-green
    '#17becf'   # blue-teal
]

extended_colors = original_colors + generate_similar_colors(original_colors)

In [21]:
def create_gantt_chart(model, ships, ship_names, stations, X, W, A, F):

    # Dataframe to store information for gantt chart
    df_gantt = pd.DataFrame(columns=['Task','Start','Finish','Resource'])

    # For all ships, add periods at each station to dataframe
    for s in ships:
        for p in stations:
            df_gantt = pd.concat([df_gantt, pd.DataFrame({'Task': f'Station {p+1}', 'Start': [X[s,p].solution_value], 'Finish': [X[s,p].solution_value + A[s,p].solution_value * unloading_times[categories[s]][p] + (1-A[s,p].solution_value) + W[s,p].solution_value], 'Resource': ship_names[s]})], ignore_index=True)

    # Get finishing time
    finish = int(max([F[s].solution_value for s in ships]))

    # Create gantt chart
    fig = ff.create_gantt(df_gantt, index_col='Resource', show_colorbar=True, bar_width=0.4, group_tasks=True, title=f'Gantt Chart for Instance 4; Finished after {finish} periods; Objective value: {int(model.objective_value)}', colors=extended_colors)
    fig.update_layout(xaxis_type='linear', autosize=False, width=1000)

    # Return gantt chart
    return fig

In [23]:
# Create gantt chart
gantt = create_gantt_chart(model, ships, ship_names, stations, X, W, A, F)

display(gantt)

C:\Users\mogra\AppData\Local\Temp\ipykernel_20472\4046311883.py:9: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

